In [1]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["all-docs"]==0.12.5
# NOTE: you may also upgrade to the latest version with the command below,
#       though a more recent version of unstructured will not have been tested with this notebook
# %pip install -q --upgrade unstructured
# Install required libraries
!pip install faiss-cpu
!pip install sentence_transformers
!pip install nltk sentence-transformers faiss-cpu numpy pandas transformers
!mkdir -p example-docs

import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import os
import nltk
import json
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
from transformers import pipeline
import faiss
from nltk.translate.bleu_score import sentence_bleu
from functools import lru_cache
from unstructured.partition.pdf import partition_pdf
from collections import Counter
from google.colab import files
from collections import Counter
import json






/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Extracting pdf to txt file

extracting articles and faqs from pdf to txt using Unstructured library

In [2]:

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_en')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading averaged_perceptron_tagger_en: Package
[nltk_data]     'averaged_perceptron_tagger_en' not found in index


False

### PDF Parsing

main strategy is extracting text so use "fast"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Define the Google Drive folder path where PDFs are stored
drive_folder_path = "/content/drive/MyDrive/pdfExtract"

# Get a list of all PDF files in the folder
pdf_files = [os.path.join(drive_folder_path, file) for file in os.listdir(drive_folder_path) if file.endswith(".pdf")]

# Process each PDF using a loop
all_elements = {}
for pdf in pdf_files:
    elements = partition_pdf(pdf)
    elements_fast = partition_pdf(pdf, strategy="fast")
    all_elements[pdf] = {"elements": elements, "elements_fast": elements_fast}

# Example: Access elements from the first PDF
if pdf_files:
    first_pdf_elements = all_elements[pdf_files[0]]["elements"]
    first_pdf_fast_elements = all_elements[pdf_files[0]]["elements_fast"]

    # Print summary of elements for one PDF
    print(f"Processed {pdf_files[0]}:")
    print(f"Standard Elements: {len(first_pdf_elements)} chunks")
    print(f"Fast Strategy Elements: {len(first_pdf_fast_elements)} chunks")
else:
    print("No PDF files found in the specified folder.")

Processed /content/drive/MyDrive/pdfExtract/AAintro.pdf:
Standard Elements: 248 chunks
Fast Strategy Elements: 248 chunks


In [5]:
display(Counter(type(element) for element in elements))
print("")
# The composition of elements can be different for elements derived with the "fast" strategy
display(Counter(type(element) for element in elements_fast))

Counter({unstructured.documents.elements.Title: 172,
         unstructured.documents.elements.Text: 4,
         unstructured.documents.elements.Footer: 15,
         unstructured.documents.elements.NarrativeText: 50,
         unstructured.documents.elements.Header: 16,
         unstructured.documents.elements.ListItem: 7})

Counter({unstructured.documents.elements.Title: 172,
         unstructured.documents.elements.Text: 4,
         unstructured.documents.elements.Footer: 15,
         unstructured.documents.elements.NarrativeText: 50,
         unstructured.documents.elements.Header: 16,
         unstructured.documents.elements.ListItem: 7})

display the type and text of some of the elements in the document:

In [6]:
display(*[(type(element), element.text) for element in elements[10:13]])

(unstructured.documents.elements.Title, 'SEE PROFILE')

(unstructured.documents.elements.Footer,
 'All content following this page was uploaded by Matthew Thompson on 11 September 2020.')

(unstructured.documents.elements.Footer,
 'The user has requested enhancement of the downloaded file.')

the parser also successfully differentiated between titles and narrative text from a PDF file. However, be aware that element classification is improving as the library evolves, tends to be more accurate with the "hi_res" strategy, and may not always correct.

Now we can join the elements and print the extracted texts from the PDF

In [7]:
print("\n\n".join([str(el) for el in elements]))

See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/344201928

Social Economy and Social Enterprise

Article · April 2020

DOI: 10.1016/B978-0-08-102295-5.10702-4

CITATIONS 3

READS 3,342

1 author:

Matthew Thompson

University College London

48 PUBLICATIONS 743 CITATIONS

SEE PROFILE

All content following this page was uploaded by Matthew Thompson on 11 September 2020.

The user has requested enhancement of the downloaded file.

Author's personal copy

Provided for non-commercial research and educational use.

Not for reproduction, distribution or commercial use.

This article was originally published in International Encyclopedia of Human Geography, 2nd Edition, published by Elsevier, and the attached copy is provided by Elsevier for the author's benefit and for the benefit of the author's institution, for non-commercial research and educational use, including without limitation, use in instruction at your institution, sen

In [8]:
# join the elements and print the extracted texts from the PDF
extracted_text = "\n\n".join([str(el) for el in elements])

# Save the extracted text to a file in your Google Drive
with open('/content/extracted_text.txt', 'w') as f:
  f.write(extracted_text)

print("Extracted text saved to /content/extracted_text.txt")

# Optionally, download the file to your local machine
# files.download('/content/drive/MyDrive/extracted_text.txt')

Extracted text saved to /content/extracted_text.txt


#Task 1: Data Ingestion and Cleaning

In [38]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    """
    Preprocess the input text:
    - Remove stopwords
    - Remove punctuation
    - Tokenize sentences
    """
    stop_words = set(stopwords.words('english'))
    translator = str.maketrans('', '', string.punctuation)

    sentences = sent_tokenize(text)  # Tokenize into sentences
    cleaned_sentences = []

    for sentence in sentences:
        # Tokenize the sentence into words and filter stopwords/punctuation
        words = word_tokenize(sentence)
        filtered_words = [word.lower().translate(translator) for word in words if word.lower() not in stop_words]
        cleaned_sentences.append(" ".join(filtered_words))

    return cleaned_sentences

def chunk_sentences(sentences, max_tokens=300, overlap=50):
    """
    Chunk the sentences into smaller sections, ensuring the chunk size is within the max_tokens limit
    and overlapping by a specified number of tokens.
    """
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = len(sentence.split())
        if current_tokens + sentence_tokens > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:]  # Maintain overlap
            current_tokens = sum(len(s.split()) for s in current_chunk)

        current_chunk.append(sentence)
        current_tokens += sentence_tokens

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def preprocess_file(file_path, doc_name):
    """
    Preprocess a single file and associate it with a document name, then chunk the sentences.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File '{file_path}' not found.")

    with open(file_path, 'r') as f:
        text = f.read()

    cleaned_sentences = preprocess_text(text)
    doc_chunks = chunk_sentences(cleaned_sentences)  # Split the sentences into chunks

    return {doc_name: doc_chunks}



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
if __name__ == "__main__":
    file_path = "/content/extracted_text.txt"  # Path to the input file

    if os.path.exists(file_path):
        doc_name = os.path.basename(file_path).split('.')[0]  # Use file name (without extension) as doc_name
        try:
            # Preprocess the file and get cleaned and chunked sentences
            processed_data = preprocess_file(file_path, doc_name)

            # Save the chunks to a JSON file
            output_path = "chunks.json"
            with open(output_path, "w") as f:
                json.dump(processed_data, f, indent=4)

            print(f"Preprocessing complete. Results saved to '{output_path}'.")
            print(f"Number of chunks: {len(processed_data[doc_name])}")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")

Preprocessing complete. Results saved to 'chunks.json'.
Number of chunks: 228


 preparing text data for further processing in NLP tasks. It cleans and divides the text into smaller, manageable chunks that can be easily used for tasks like information retrieval, embedding generation, or document classification. The chunking process with overlap ensures that the context is preserved across chunks, which is important for maintaining the semantic integrity of the content.

# Task 2: Embedding Creation and Retrieval

In [40]:


def create_embeddings(data, model_name):
    model = SentenceTransformer(model_name)
    embeddings = []

    for doc, chunks in data.items():
        for idx, chunk in enumerate(chunks):
            vector = model.encode(chunk)
            embeddings.append({
                "document": doc,
                "chunk_index": idx,
                "chunk_text": chunk,
                "embedding": vector.tolist()
            })

    return embeddings

if __name__ == "__main__":
    with open("chunks.json", "r") as f:
        data = json.load(f)

    model_1 = "distilbert-base-nli-stsb-mean-tokens"
    model_2 = "all-MiniLM-L6-v2"

    embeddings_1 = create_embeddings(data, model_1)
    embeddings_2 = create_embeddings(data, model_2)

    # Save embeddings
    pd.DataFrame(embeddings_1).to_csv("embeddings_model_1.csv", index=False)
    pd.DataFrame(embeddings_2).to_csv("embeddings_model_2.csv", index=False)


In [12]:
def load_embeddings(file):
    """
    Load embeddings and metadata from a CSV file.
    """
    if not os.path.exists(file):
        raise FileNotFoundError(f"File not found: {file}")
    try:
        data = pd.read_csv(file)
        if data.empty:
            raise pd.errors.EmptyDataError("The CSV file is empty.")
        vectors = np.array([eval(v) for v in data["embedding"]])
        return vectors, data
    except pd.errors.EmptyDataError as e:
        print(f"Error loading embeddings: {e}")
        return np.array([]), pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

def create_faiss_index(embeddings):
    """
    Create a FAISS index for fast similarity search using cosine similarity.
    """
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    dimension = normalized_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(normalized_embeddings)
    return index

@lru_cache(maxsize=100)
def encode_query(query, model):
    """
    Encode a query using the embedding model with caching.
    """
    query_vector = model.encode(query).reshape(1, -1)
    return query_vector / np.linalg.norm(query_vector, axis=1, keepdims=True)

def retrieve_chunks(query, model, index, data, top_k=3):
    """
    Retrieve the top-k most similar chunks for a given query.
    """
    query_vector = encode_query(query, model)
    distances, indices = index.search(query_vector, top_k)
    results = []
    for idx in indices[0]:
        if idx < len(data):
            results.append(data.iloc[idx].to_dict())
    return results

def batch_query_retrieval(queries, model, index, data, top_k=3):
    """
    Retrieve results for a batch of queries.
    """
    results = []
    for query in queries:
        results.append(retrieve_chunks(query, model, index, data, top_k))
    return results

def log_performance(start_time, end_time, task_name):
    """
    Log performance metrics for a specific task.
    """
    duration = end_time - start_time
    print(f"{task_name} completed in {duration:.4f} seconds.")



### DISTILBERT MODELLING


In [20]:
import time
if __name__ == "__main__":
    file = "embeddings_model_1.csv"
    try:
        # Load embeddings and metadata
        start_time = time.time()
        embeddings, metadata = load_embeddings(file)
        log_performance(start_time, time.time(), "Loading embeddings")

        if embeddings.size == 0:
            print("Embeddings are empty. Please check the input file.")
            exit(1)

        # Create FAISS index
        start_time = time.time()
        faiss_index = create_faiss_index(embeddings)
        log_performance(start_time, time.time(), "Creating FAISS index")

        # Load SentenceTransformer model
        start_time = time.time()
        model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")
        log_performance(start_time, time.time(), "Loading embedding model")

        # Example query
        query = "What is the company's mission?"
        start_time = time.time()
        results = retrieve_chunks(query, model, faiss_index, metadata)
        log_performance(start_time, time.time(), "Single query retrieval")

        # Output the results
        print("Top-3 Retrieved Chunks and Metadata:")
        for result in results:
            print(result)

        # Batch queries
        batch_queries = [
            "What is the company's social economy mission?",
            "How does the mission operate?",
            "What are the future action plans"
        ]
        start_time = time.time()
        batch_results = batch_query_retrieval(batch_queries, model, faiss_index, metadata)
        log_performance(start_time, time.time(), "Batch query retrieval")

        print("Batch Query Results:")
        for i, res in enumerate(batch_results):
            print(f"Query {i+1}: {batch_queries[i]}")
            for chunk in res:
                print(chunk)

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Loading embeddings completed in 0.5752 seconds.
Creating FAISS index completed in 0.0009 seconds.
Loading embedding model completed in 0.5533 seconds.
Single query retrieval completed in 0.0980 seconds.
Top-3 Retrieved Chunks and Metadata:
{'document': 'cleaned_sentences', 'chunk_index': 60, 'chunk_text': 'put simply  social economy produces goods services via entrepreneurial risktaking  like capitalist economy  associative collective organization mutual public interest rather proﬁt  like public sector   asso ciated number synonyms  “ third sector  ” “ civil society  ” “ nonproﬁt sector  ” “ notforproﬁt sector  ” “ voluntary sector  ” “ community economy  ” “ solidarity economy  ” among others  related terms  social economy distinctd broadly describing aspects economy notforproﬁt  remain outside market state  create economic social value  social purpose  purpose  broadly deﬁned  reasserting social justice prin ciples economydchallenging social exclusion developing systems inclusion  pr

### MINI-LM

In [41]:
import time
if __name__ == "__main__":
    file = "embeddings_model_2.csv"
    try:
        # Load embeddings and metadata
        start_time = time.time()
        embeddings, metadata = load_embeddings(file)
        log_performance(start_time, time.time(), "Loading embeddings")

        if embeddings.size == 0:
            print("Embeddings are empty. Please check the input file.")
            exit(1)

        # Create FAISS index
        start_time = time.time()
        faiss_index = create_faiss_index(embeddings)
        log_performance(start_time, time.time(), "Creating FAISS index")

        # Load SentenceTransformer model
        start_time = time.time()
        model = SentenceTransformer("all-MiniLM-L6-v2")
        log_performance(start_time, time.time(), "Loading embedding model")

        # Example query
        query = "What is the company's mission?"
        start_time = time.time()
        results = retrieve_chunks(query, model, faiss_index, metadata)
        log_performance(start_time, time.time(), "Single query retrieval")

        # Output the results
        print("Top-3 Retrieved Chunks and Metadata:")
        for result in results:
            print(result)

        # Batch queries
        batch_queries = [
            "What is the company's social economy mission?",
            "How does the mission operate?",
            "What are the future action plans"
        ]
        start_time = time.time()
        batch_results = batch_query_retrieval(batch_queries, model, faiss_index, metadata)
        log_performance(start_time, time.time(), "Batch query retrieval")

        print("Batch Query Results:")
        for i, res in enumerate(batch_results):
            print(f"Query {i+1}: {batch_queries[i]}")
            for chunk in res:
                print(chunk)

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Loading embeddings completed in 0.3018 seconds.
Creating FAISS index completed in 0.0014 seconds.
Loading embedding model completed in 0.5482 seconds.
Single query retrieval completed in 0.0301 seconds.
Top-3 Retrieved Chunks and Metadata:
{'document': 'extracted_text', 'chunk_index': 183, 'chunk_text': 'question reﬂected parallel largely separate evolution distinct network regional national infrastruc ture organizations supporting development organizational type  charities  coops  social enterprises   united kingdom split older councils voluntary service  known cvss   cooperative development societies recently established social enterprise networksdeach proud traditions  membership bases  funding sour ces  political allies  operating ethos  challenge creation cohesive social economy bring often competing traditions together forge political alliances  pool resources  share learning  often far greater integration coordination infrastructure organizations across national borders within m

The optimization of the script focuses on enhancing performance and scalability through several key strategies. First, Batch Query Retrieval was implemented using the batch_query_retrieval() function, allowing the efficient processing of multiple queries simultaneously, thereby reducing overhead for repeated operations. Additionally, Caching was introduced via the @lru_cache decorator, which stores the results of encoded queries to accelerate the retrieval process for frequently used queries. To monitor and analyze performance, a Performance Logging mechanism was added, utilizing the log_performance() function to measure and report the time taken for critical tasks, helping identify bottlenecks. Finally, the solution scales effectively by normalizing embeddings and leveraging FAISS's capability to handle large datasets efficiently, coupled with batching techniques to manage high query volumes seamlessly. These optimizations collectively improve the script's responsiveness, scalability, and reliability.

# Task 3: Contextual Query Handling and RAG

In [21]:
# Query history to handle multi-turn queries
query_history = []

def load_embeddings(file):
    """
    Load embeddings and metadata from a CSV file.
    """
    data = pd.read_csv(file)
    vectors = np.array([eval(v) for v in data["embedding"]])  # Convert strings to numpy arrays
    return vectors, data

def create_faiss_index(embeddings):
    """
    Create a FAISS index for fast retrieval of similar vectors.
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

def retrieve_chunks(query, model, index, data, top_k=3):
    """
    Retrieve the top-k most similar chunks using cosine similarity.
    """
    query_vector = model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    results = [data.iloc[idx].to_dict() for idx in indices[0]]
    return results

def contextual_retrieval(query, model, index, data, top_k=3):
    """
    Handle multi-turn queries by combining query history.
    """
    global query_history
    query_history.append(query)
    combined_query = " ".join(query_history)
    return retrieve_chunks(combined_query, model, index, data, top_k)

def generate_response(context_chunks, query, generation_pipeline):
    """
    Generate a response using a Hugging Face transformer pipeline.
    """
    context = " ".join([chunk["chunk_text"] for chunk in context_chunks])
    # Limit the context length to prevent exceeding model's input size
    context = context[:512]  # Adjust the value as needed
    prompt = f"Context: {context}\n\nQuery: {query}\n\nAnswer:"
    # Use max_new_tokens instead of max_length to specify the output length
    response = generation_pipeline(prompt, max_new_tokens=100, do_sample=True, top_k=50)
    return response[0]["generated_text"]



### DISTILBERT

In [45]:
if __name__ == "__main__":
    # Load embeddings and metadata
    embeddings, metadata = load_embeddings("embeddings_model_1.csv")
    faiss_index = create_faiss_index(embeddings)

    # Initialize models
    retrieval_model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")
    generation_pipeline = pipeline("text-generation", model="gpt2")

    # Example query
    query = "What is social economy?"
    context_chunks = contextual_retrieval(query, retrieval_model, faiss_index, metadata)
    response = generate_response(context_chunks, query, generation_pipeline)
    print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Context: reappropriating new economy across scales social economy part   turn “ social ” across multiple societal areasdfrom social media social networking social housing social policy social entrepreneurship social innovation  like economy whole  social economy made individuals  ﬁrms  processes order economic relations structure economic activity  micro scale individual  rather entrepreneurs investors  social economy see social entrepreneurs social investors  meso scale ﬁrm  social enterprises  macro scale system s

Query: What is social economy?

Answer: The social economy's social structure.

Social Economic Activity

  𝬑 郞徭   ༻  𝯀 1 ﾨ ﵿ ﵏  8 ﻑ ﺽ 10    𝯀 ﯀ 30 �


test the model by changing the question in query

### MiniLM

In [44]:
if __name__ == "__main__":
    # Load embeddings and metadata
    embeddings, metadata = load_embeddings("embeddings_model_2.csv")
    faiss_index = create_faiss_index(embeddings)

    # Initialize models
    retrieval_model = SentenceTransformer("all-MiniLM-L6-v2")
    generation_pipeline = pipeline("text-generation", model="gpt2")

    # Example query
    query = "What is social economy?"
    context_chunks = contextual_retrieval(query, retrieval_model, faiss_index, metadata)
    response = generate_response(context_chunks, query, generation_pipeline)
    print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Context: put simply  social economy produces goods services via entrepreneurial risktaking  like capitalist economy  associative collective organization mutual public interest rather proﬁt  like public sector   asso ciated number synonyms  “ third sector  ” “ civil society  ” “ nonproﬁt sector  ” “ notforproﬁt sector  ” “ voluntary sector  ” “ community economy  ” “ solidarity economy  ” among others  related terms  social economy distinctd broadly describing aspects economy notforproﬁt  remain outside market state 

Query: What is social economy?

Answer:

Social economy is the collective collective purpose of the means used to produce an asset or a service in a market context and includes not for the benefit of one's own personal economy the individual economy, but one's own interests and preferences.

The social economy of self-interest is not the economic condition within which the self-interests of individuals make sense, but rather is the objective nature of social life and its c

test the model by changing the question in query